In [1]:
import tensorflow as tf
import numpy as np
import cv2
import argparse
import os
import re
from IPython.display import clear_output, Image, display, HTML
from moviepy.editor import VideoFileClip
from pyheatmap.heatmap import HeatMap

/home/carnd/anaconda3/envs/carnd-term1/lib/python3.5/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [7]:
def heatmap(den, img):
    den_resized = np.zeros((den.shape[0] * 4, den.shape[1] * 4))
    for i in range(den_resized.shape[0]):
        for j in range(den_resized.shape[1]):
            den_resized[i][j] = den[int(i / 4)][int(j / 4)] / 16
    den = den_resized
    
    count = np.sum(den)
    den = den * 10 / np.max(den)
    
    w = img.shape[1]
    h = img.shape[0]
     
    data = []
    for j in range(len(den)):
        for i in range(len(den[0])):
            for k in range(int(den[j][i])):
                data.append([i + 1, j + 1])
    hm = HeatMap(data, base = './test/img.jpg')
    return hm

In [11]:
font = cv2.FONT_HERSHEY_SIMPLEX
with tf.Session() as sess:
        feed_vid = cv2.VideoCapture("crowd10.mp4")
        success = True
        if success:
            success, im = feed_vid.read()
            #print("success-", success)
            new_saver = tf.train.import_meta_graph("./model.ckpt.meta")
            new_saver.restore(sess, tf.train.latest_checkpoint("./"))
            graph = tf.get_default_graph()
            op_to_restore = graph.get_tensor_by_name("add_12:0")
            x = graph.get_tensor_by_name('Placeholder:0')
            fps = feed_vid.get(cv2.CAP_PROP_FPS)
            fps = np.int32(fps)
            print("Frames Per Second:",fps,"\n")
            counter = 0
            avg = 0
            #imlist = []
            file = open("Count_per_sec_Vid8_1.txt", "w")
            if success:
                out = cv2.VideoWriter('output_vid.mp4',cv2.VideoWriter_fourcc('M','J','P','G'), fps, (im.shape[0],im.shape[1]))
            cc = None
            while success: 
                counter +=1
                cv2.imwrite('./test/img.jpg',im)
                img = np.copy(im)
                img = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
                img = np.array(img)
                img = (img - 127.5) / 128
                #print(img.shape)
                x_in = np.reshape(img, (1, img.shape[0], img.shape[1], 1))
                x_in = np.float32(x_in)
                y_pred = []
                y_pred = sess.run(op_to_restore, feed_dict={x: x_in})
                sum =np.absolute(np.int32(np.sum(y_pred)))
                '''if sum - 15 <= 0:
                    sum += 20'''
                y_p_den = np.reshape(y_pred, (y_pred.shape[1], y_pred.shape[2]))
                hm = heatmap(y_p_den, im)
                hm.heatmap(save_as = './test/img2.jpg')
                frame = cv2.imread('./test/img2.jpg')
                if not cc is None:
                    text=str(cc)+' people detected'
                    cv2.putText(hm, text, (10, 10), font, 0.5, (0, 0, 0), 2, cv2.LINE_AA)
                if counter<=fps:
                    avg += sum
                    out.write(frame)
                else:
                    counter =0 
                    avg = np.int32(avg/fps)
                    print(avg)
                    cc=avg
                    file.write(str(avg))
                    file.write("\n")
                    avg = 0
                    avg += sum
                    out.write(frame)
                #count = "Crowd Count - " + str(sum)
                #counter += 1
                #cv2.putText(im, count, (0, 0), font, 0.5, (0, 0, 0), 2, cv2.LINE_AA)
                #imlist.append(im)
                '''if counter==10:
                    for e in imlist:
                        cv2.imshow('Crowd Count', im)
                    imlist = []
                    counter = 0'''
                success, im = feed_vid.read()
        file.close()
        feed_vid.release()
        out.release()
        cv2.destroyAllWindows()

INFO:tensorflow:Restoring parameters from ./model.ckpt
Frames Per Second: 30 



KeyboardInterrupt: 